Enlace a tu repositorio de GitHub: [Wilmer Marquez](https://github.com/wilmer-marquez?tab=repositories)

## Descripcion de las columnas

**mag:** Magnitud en escala richter [0-10]

**place:** descripcion textual de la region geografica del evento 

**time:** datetime del evento

**felt:** numero de reportes de sensacion del evento

**cdi:** intensidad maxima reportada del evento[0.0-10.0]

**alert:** nivel de alerta del evento: "green","yellow","orange","red"

**tsunami:** establece dos valores, 1 para grandes eventos y 0 para lo contrario. El 0 no indica si existio o existira, pero determina que no habra grandes riesgos 

**sig:** describe cuan significativo fue el evento [0-1000]

**nst:** numero de estaciones que informaron el evento

**mgType:** metodo del algoritmo usado para medir la magnitud, estas pueden ser: "Md","Ml","Ms","Mw","Me","Mi","Mb","Mlg"

**longitud:** grados decimales de longitud, valores negativos para longitudes occidentales[-180.0 - 180.0]

**latitud:** grados decimales latitud, valores negativos para latitudes del sur[-90.0 - 90.0]

**profundidad:** profundida maxima en kilometros del evento

# Extracción

In [ ]:
import pandas as pd
import requests as req
import numpy as np
import folium #libreria para hacer mapas
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# Matplotlib inline para visualizar los gráficos de Matplotlib
#Para que se vean los gráficos en el notebook
%matplotlib inline 
#Para que se vean los gráficos en alta resolución
%config Inlinebackend.figure_format= 'retina' 
# Configuración para setear y que todas las fig de Seaborn salgan por defecto con este tamaño
# Se puede especificar el tamaño de cada figura
sns.set_context("poster") #paper, notebook, talk, poster
sns.set(rc={"figure.figsize": (12.,10.)}) #Tamaño de las figuras
sns.set_style("darkgrid") #darkgrid, whitegrid, dark, white, ticks

In [ ]:
URL_BASE='https://earthquake.usgs.gov/fdsnws/event/1/'

In [ ]:
busqueda='/query?format=geojson&starttime=2023-06-01&endtime=2023-07-05'

In [ ]:
URL_BASE+busqueda

In [ ]:
response=req.get(URL_BASE+busqueda)

response

In [ ]:
terre = response.json()['features']
terre

In [ ]:
terremoto = pd.DataFrame(terre)
terremoto

# Limpieza

In [ ]:
terremoto = terremoto.drop(columns=['id','type'])
terremoto

In [ ]:
terremoto.geometry.iloc[0]

In [ ]:
terremoto.properties.iloc[0]

In [ ]:
terro1 = terremoto.properties
terro1

In [ ]:
terro2 = terremoto.geometry
terro2

In [ ]:
terremo1 = pd.json_normalize(terro1)
terremo1

In [ ]:
terremo2 = pd.json_normalize(terro2)
terremo2

In [ ]:
terremo2[['longitud', 'latitud', 'profundidad']] = pd.DataFrame(terremo2['coordinates'].tolist())
terremo2

In [ ]:
terremo2 = terremo2.drop(columns=['coordinates','type'])
terremo2

In [ ]:
terremo1.sample(5)

In [ ]:
terremo1 = terremo1.drop(columns=['url','updated','detail','ids','title','gap','dmin','mmi','type','code','status','net',
                                  'rms','sources','types','tz'])
terremo1.sample(2)

In [ ]:
terremoto = pd.concat([terremo1, terremo2], axis=1)
terremoto

In [ ]:
terremoto.time = terremoto.time.astype('datetime64[ns]')
terremoto

In [ ]:
terremoto.time = terremoto['time'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S.%f")[:-3])
terremoto

In [ ]:
terremoto['ciudad'] = terremoto.place.str.split(',').str.get(1).str.strip()
terremoto

# Análisis

In [ ]:
terremoto.info()

In [ ]:
terremoto.describe().T

In [ ]:
terremoto.isna().sum()

In [ ]:
terremoto = terremoto.drop(columns=['felt','cdi','alert'])
terremoto.sample(5)

In [ ]:
sns.boxplot(terremoto)

In [ ]:
corr = terremoto.corr(numeric_only=True)
mascara = np.triu(terremoto,1)
mascara = np.triu(np.ones_like(corr, dtype=bool)) # generamos la máscara para la triangular superior
color_map = sns.diverging_palette(220, 20, as_cmap=True) # Paleta de colores
sns.heatmap(corr,  
            mask = mascara,
            cmap=color_map,
            square=True, #que los datos se vean como cuadrados
            linewidth=0.5, #ancho de línea
            vmax=1,
            vmin=0,
            cbar_kws={"shrink": .5
                     },# barra lateral
            annot=True #Anotacion, que te ponga el valor de la corr() dentro de la casilla
);

In [ ]:
terremoto.sample(1)

In [ ]:
ter1 = terremoto.groupby('magType').agg({'mag':'count'}).reset_index()
ter1

In [ ]:
fig = px.bar(ter1,x='magType',y='mag',text_auto=True)
fig.show()

In [ ]:
ter2 = terremoto[terremoto['tsunami'] == 1].groupby('ciudad').agg({'tsunami': 'count'}).sort_values('tsunami', ascending=False)[0:10].reset_index()
ter2

In [ ]:
fig = px.bar(ter2,x='ciudad',y='tsunami',text_auto=True)
fig.show()

In [ ]:
ter3 = terremoto[terremoto['mag'] > 5].groupby('ciudad').agg({'mag': 'count'}).sort_values('mag', ascending=False)[0:10].reset_index()
ter3

In [ ]:
fig = px.scatter(ter3,x='ciudad',y='mag',size='mag')
fig.show()

In [ ]:
ter4 = terremoto[['longitud','latitud']].mean().reset_index()# hacer histograma de latitud y longitud
ter4

In [ ]:
ter5 = terremoto.groupby('ciudad').agg({'sig':'mean'}).sort_values('sig', ascending=False)[0:10].round(2).reset_index()
ter5

In [ ]:
fig = px.bar(ter5,x='ciudad',y='sig',text_auto=True)
fig.show()

In [ ]:
terremoto['time'] = pd.to_datetime(terremoto['time'])
terremoto['tiempo'] = terremoto['time'].dt.time
terremoto

In [ ]:
terremoto2 = terremoto[['latitud', 'longitud','tiempo']]
terremoto2

In [ ]:
mapa = folium.Map(location=[terremoto2['latitud'][1], terremoto2['longitud'][2]], zoom_start=2, )
for i, row in terremoto2.iterrows():
    if i == 0: #punto donde empiezo a trackear la ISS
        folium.Marker(location=[row['latitud'], row['longitud']], tooltip = row['tiempo'], icon=folium.Icon(color='green')).add_to(mapa)
    elif i == len(terremoto2)-1: #punto final del recorrido
        folium.Marker(location=[row['latitud'], row['longitud']], tooltip = row['tiempo'], icon=folium.Icon(color='red')).add_to(mapa)
    else:    
        folium.Marker(location=[row['latitud'], row['longitud']], tooltip = row['tiempo']).add_to(mapa)
mapa